In [ ]:
##热电联产机组的功率和热力约束函数设置
def get_chp_bounds(chp_units):
    """
    根据CHP机组参数定义功率和热力约束函数
    返回一个字典，包含每个机组的约束函数
    """
    bounds = {}
    
    # 遍历所有CHP机组
    for unit_id in chp_units['Units']:
        region = chp_units['Feasible_Region'][int(unit_id)]
        
        # 根据机组ID设置不同的约束函数
        if unit_id in [14, 16]:  # 第1类CHP机组
            bounds[f'unit{unit_id}'] = {
                'P_min': lambda h: jnp.maximum(98.8, 81 + (215 - 81) * h / 180),
                'P_max': lambda h: jnp.minimum(247, 215 + (247 - 215) * (180 - h) / 180),
                'H_min': lambda p: 0,
                'H_max': lambda p: jnp.where(
                    p < 215,
                    180 * (p - 81) / (215 - 81),
                    180 * (247 - p) / (247 - 215)
                )
            }
        elif unit_id in [15, 17]:  # 第2类CHP机组
            bounds[f'unit{unit_id}'] = {
                'P_min': lambda h: jnp.maximum(44, 40 + (110.2 - 40) * h / 135.6),
                'P_max': lambda h: jnp.minimum(125.8, 110.2 + (125.8 - 110.2) * (135.6 - h) / 135.6),
                'H_min': lambda p: 0,
                'H_max': lambda p: jnp.where(
                    p < 110.2,
                    135.6 * (p - 40) / (110.2 - 40),
                    135.6 * (125.8 - p) / (125.8 - 110.2)
                )
            }
        elif unit_id == 18:  # 第3类CHP机组
            bounds[f'unit{unit_id}'] = {
                'P_min': lambda h: jnp.maximum(20, 10 + (45 - 10) * h / 55),
                'P_max': lambda h: jnp.minimum(60, 45 + (60 - 45) * (55 - h) / 55),
                'H_min': lambda p: 0,
                'H_max': lambda p: jnp.where(
                    p < 45,
                    55 * (p - 10) / (45 - 10),
                    55 * (60 - p) / (60 - 45)
                )
            }
        elif unit_id == 19:  # 第4类CHP机组
            bounds[f'unit{unit_id}'] = {
                'P_min': lambda h: jnp.maximum(35, 35 + (90 - 35) * h / 45),
                'P_max': lambda h: jnp.minimum(105, 90 + (105 - 90) * (45 - h) / 45),
                'H_min': lambda p: 0,
                'H_max': lambda p: jnp.where(
                    p < 90,
                    45 * (p - 35) / (90 - 35),
                    45 * (105 - p) / (105 - 90)
                )
            }
    
    return bounds

# 存储约束函数而不是固定值
self.chp_bounds = get_chp_bounds(chp_units)

In [ ]:
## 上一版本
def get_chp_bounds(chp_units):
            """
            定义热电联产机组的功率和热力约束函数
            返回一个字典,包含每个机组的约束函数
            """
            bounds = {
                'unit2': {
                    'P_max': lambda h: (11115 - 8 * h) / 45,  # 功率上限是热力的函数
                    'P_min': lambda h: 0,  # 功率下限
                    'H_max': lambda p: jnp.minimum((11115 - 45 * p) / 8,  # 热力上限是功率的函数
                                                 jnp.abs((7952 - 75.2 * p) / 134)),
                    'H_min': lambda p: 0   # 热力下限
                },
                'unit3': {
                    'P_max': lambda h: (11115 - 8 * h) / 45,
                    'P_min': lambda h: 0,
                    'H_max': lambda p: jnp.minimum((11115 - 45 * p) / 8, 
                                                 jnp.abs((7952 - 75.2 * p) / 134)),
                    'H_min': lambda p: 0
                }
            }
            return bounds

In [ ]:
# 对热电联产机组进行热能约束
    def update_chp_unit_heat(self,power_chp, heat, index):
        #根据对应的热点机组功率来调整其热能输出
        p = power_chp[index]
        
        # CHP unit 14 & 16
        if index in [0, 2]:
            if p >= 81 and p <= 98.8:
                lower_bound = 104.8 - 524 / 89 * (p - 81)
                upper_bound = 104.8 + 188 / 335 * (p - 81)
            elif p > 98.8 and p <= 215:
                lower_bound = 0
                upper_bound = 104.8 + 188 / 335 * (p - 81)
            elif p > 215 and p <= 247:
                lower_bound = 0
                upper_bound = -45 / 8 * (p - 247)
            else:
                lower_bound = 0
                upper_bound = 0
                    
            heat = heat.at[index].set(jnp.clip(heat[index], lower_bound, upper_bound))
                
        # CHP unit 15 & 17
        if index in [1, 3]:
            if p >= 40 and p <= 44:
                lower_bound = 75 - 591 / 40 * (p - 40)
                upper_bound = 75 + 101 / 117 * (p - 40)
            elif p > 44 and p <= 110.2:
                lower_bound = 0
                upper_bound = 75 + 101 / 117 * (p - 40)
            elif p > 110.2 and p <= 125.8:
                lower_bound = 0
                upper_bound = 32.4 - 86 / 13 * (p - 125.8)
            else:
                lower_bound = 0
                upper_bound = 0
                    
            heat = heat.at[index].set(jnp.clip(heat[index], lower_bound, upper_bound))
            
        # CHP unit 18
        if index == 4:
            if p >= 10 and p <= 20:
                lower_bound = 40 - 4 * (p - 10)
                upper_bound = 40 + 3/7 * (p - 10)
            elif p > 20 and p <= 45:
                lower_bound = 0
                upper_bound = 40 + 3/7 * (p - 10)
            elif p > 45 and p <= 60:
                lower_bound = 0
                upper_bound = -11/3 * (p - 60)
            else:
                lower_bound = 0
                upper_bound = 0
                
            heat = heat.at[index].set(jnp.clip(heat[index], lower_bound, upper_bound))
            
        # CHP unit 19
        if index == 5:
            if p >= 35 and p <= 90:
                lower_bound = 0
                upper_bound = 20 + 5/11 * (p - 35)
            elif p > 90 and p <= 105:
                lower_bound = 0
                upper_bound = -5/3 * (p - 105)
            else:
                lower_bound = 0
                upper_bound = 0
                
            heat = heat.at[index].set(jnp.clip(heat[index], lower_bound, upper_bound))
            
        
        # 更新并返回整个热能数组
        return heat

In [ ]:
# # 对热电联产机组进行热能约束
    # def update_chp_unit_heat(self, power_chp, heat, index):
    #     """
    #     更新热电联产机组的热能输出
    #     Args:
    #         power_chp: 热电联产机组的功率输出 (dim_total,)
    #         heat: 热能输出
    #         index: 当前处理的机组索引
    #     Returns:
    #         更新后的热能输出
    #     """
    #     # 获取对应索引的功率值，并确保是float32类型
    #     p = power_chp[index].astype(jnp.float32)

    #     # 定义用于 CHP 单元 14 & 16 的处理逻辑
    #     def chp_14_16(_):
    #         cond1 = (p >= 81) & (p <= 98.8)
    #         cond2 = (p > 98.8) & (p <= 215)
    #         cond3 = (p > 215) & (p <= 247)

    #         lower_bound = lax.cond(
    #             cond1, 
    #             lambda _: 104.8 - 524 / 89 * (p - 81),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: jnp.float32(0.0),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: jnp.float32(0.0),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
            
    #         upper_bound = lax.cond(
    #             cond1,
    #             lambda _: 104.8 + 188 / 335 * (p - 81),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: 104.8 + 188 / 335 * (p - 81),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: -45 / 8 * (p - 247),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
    #         return jnp.clip(heat[index].astype(jnp.float32), lower_bound, upper_bound)

    #     # 定义用于 CHP 单元 15 & 17 的处理逻辑
    #     def chp_15_17(_):
    #         cond1 = (p >= 40) & (p <= 44)
    #         cond2 = (p > 44) & (p <= 110.2)
    #         cond3 = (p > 110.2) & (p <= 125.8)

    #         lower_bound = lax.cond(
    #             cond1, 
    #             lambda _: 75 - 591 / 40 * (p - 40),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: jnp.float32(0.0),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: jnp.float32(0.0),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
            
    #         upper_bound = lax.cond(
    #             cond1,
    #             lambda _: 75 + 101 / 117 * (p - 40),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: 75 + 101 / 117 * (p - 40),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: jnp.float32(0.0),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
    #         return jnp.clip(heat[index].astype(jnp.float32), lower_bound, upper_bound)

    #     # 定义用于 CHP 单元 18 的处理逻辑
    #     def chp_18(_):
    #         cond1 = (p >= 10) & (p <= 20)
    #         cond2 = (p > 20) & (p <= 45)
    #         cond3 = (p > 45) & (p <= 60)

    #         lower_bound = lax.cond(
    #             cond1,
    #             lambda _: 40 - 4 * (p - 10),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: jnp.float32(0.0),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: jnp.float32(0.0),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
            
    #         upper_bound = lax.cond(
    #             cond1,
    #             lambda _: 40 + 3 / 7 * (p - 10),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: 40 + 3 / 7 * (p - 10),
    #                 lambda _: lax.cond(
    #                     cond3,
    #                     lambda _: -11 / 3 * (p - 60),
    #                     lambda _: jnp.float32(0.0)
    #                 )
    #             )
    #         )
    #         return jnp.clip(heat[index].astype(jnp.float32), lower_bound, upper_bound)

    #     # 定义用于 CHP 单元 19 的处理逻辑
    #     def chp_19(_):
    #         cond1 = (p >= 35) & (p <= 90)
    #         cond2 = (p > 90) & (p <= 105)

    #         lower_bound = 0
    #         upper_bound = lax.cond(
    #             cond1,
    #             lambda _: 20 + 5 / 11 * (p - 35),
    #             lambda _: lax.cond(
    #                 cond2,
    #                 lambda _: -5 / 3 * (p - 105),
    #                 lambda _: 0,
    #                 None
    #             )
    #         )
    #         return jnp.clip(heat[index].astype(jnp.float32), lower_bound, upper_bound)

    #     # 使用 JAX 的条件函数替代 Python 的 if 语句
    #     return lax.cond(
    #         (index == 0) | (index == 2),
    #         chp_14_16,
    #         lambda _: lax.cond(
    #             (index == 1) | (index == 3),
    #             chp_15_17,
    #             lambda _: lax.cond(
    #                 (index == 4),
    #                 chp_18,
    #                 lambda _: lax.cond(
    #                     (index == 5),
    #                     chp_19,
    #                     lambda _: heat[index]
    #                 )
    #             )
    #         )
    #     )


In [ ]:
def adjust_power_heat_limitation(self,solution):
    """对三种不同的机组进行功率和热能约束"""
    power = solution[0:self.dim_P]
    power_chp = solution[self.dim_P:self.dim_P + self.dim_PH]
    power_heat = solution[self.dim_P + self.dim_PH:self.dim_P + 2*self.dim_PH]
    thermal = solution[self.dim_P + 2*self.dim_PH:]

    # 从原始解数组开始
    adjusted_solution = solution

    # 对纯电机组进行功率约束
    adjusted_solution = adjusted_solution.at[0:self.dim_P].set(jnp.clip(power, self.Pmin, self.Pmax))
        
    # 对热点联产机组进行功率约束
    power_chp = jnp.clip(power_chp, self.PRmin, self.PRmax)

    # 对热点联产机组进行热能约束
    power_heat = jax.vmap(lambda i: self.update_chp_unit_heat(power_chp, power_heat, i))(
        jnp.arange(self.dim_PH)
    )

        # 对纯热机组进行热能约束
    thermal = jnp.clip(thermal, self.Hmin, self.Hmax)

    return jnp.concatenate([power, power_chp, power_heat, thermal])

In [ ]:
## 在def adjust_power_heat_limitation(self,solution):中的调用为：

# 对联合热电机组（热功率部分）应用热能约束
adjusted_power_heat = jax.vmap(
    lambda i: self.update_chp_unit_heat(
    adjusted_power_chp, solution[power_heat_indices], i
    )
)(jnp.arange(self.dim_PH))

adjusted_solution = adjusted_solution.at[power_heat_indices].set(adjusted_power_heat)


def update_chp_unit_heat(self, power_chp, heat, index):
        """
        更新热电联产机组的热能输出

        Args:
            power_chp: 热电联产机组的功率输出 (dim_total,)
            heat: 热能输出数组
            index: 当前处理的机组索引

        Returns:
            更新后的热能输出
        """
        # 获取对应索引的功率值，并转换为 float32 类型
        p = power_chp[index].astype(jnp.float32)
        h = heat[index].astype(jnp.float32)

        # 定义计算热能输出的辅助函数
        def compute_heat(lower_bound, upper_bound):
            return jnp.clip(h, lower_bound, upper_bound)

        # 定义每个 CHP 单元的处理逻辑，修改为接受一个参数
        def chp_14_16(_):
            cond1 = (p >= 81) & (p <= 98.8)
            cond2 = (p > 98.8) & (p <= 215)
            cond3 = (p > 215) & (p <= 247)

            lower_bound = jnp.where(cond1, 104.8 - (524 / 89) * (p - 81), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                104.8 + (188 / 335) * (p - 81),
                jnp.where(cond3, - (45 / 8) * (p - 247), 0.0)
            )
            return compute_heat(lower_bound, upper_bound)

        def chp_15_17(_):
            cond1 = (p >= 40) & (p <= 44)
            cond2 = (p > 44) & (p <= 110.2)
            cond3 = (p > 110.2) & (p <= 125.8)

            lower_bound = jnp.where(cond1, 75 - (591 / 40) * (p - 40), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                75 + (101 / 117) * (p - 40),
                jnp.where(cond3, 32.4 - (86 / 13) * (p - 125.8), 0.0)
            )
            return compute_heat(lower_bound, upper_bound)

        def chp_18(_):
            cond1 = (p >= 10) & (p <= 20)
            cond2 = (p > 20) & (p <= 45)
            cond3 = (p > 45) & (p <= 60)

            lower_bound = jnp.where(cond1, 40 - 4 * (p - 10), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                40 + (3 / 7) * (p - 10),
                jnp.where(cond3, - (11 / 3) * (p - 60), 0.0)
            )
            return compute_heat(lower_bound, upper_bound)

        def chp_19(_):
            cond1 = (p >= 35) & (p <= 90)
            cond2 = (p > 90) & (p <= 105)

            lower_bound = 0.0
            upper_bound = jnp.where(
                cond1,
                20 + (5 / 11) * (p - 35),
                jnp.where(cond2, - (5 / 3) * (p - 105), 0.0)
            )
            return compute_heat(lower_bound, upper_bound)

        # 默认情况下，返回原始热能输出，接受一个参数
        def default_case(_):
            return h

        # 使用 JAX 的条件函数选择对应的处理逻辑，添加 operand 参数
        result = lax.cond(
            (index == 0) | (index == 2),
            chp_14_16,
            lambda _: lax.cond(
                (index == 1) | (index == 3),
                chp_15_17,
                lambda _: lax.cond(
                    index == 4,
                    chp_18,
                    lambda _: lax.cond(
                        index == 5,
                        chp_19,
                        default_case,
                        operand=None  # 添加 operand 参数
                    ),
                    operand=None  # 添加 operand 参数
                ),
                operand=None  # 添加 operand 参数
            ),
            operand=None  # 添加 operand 参数
        )

        return result

In [ ]:
  # 获取所有的热能输出
        total_heat = solution[self.dim_P + self.dim_PH : ]
        # 获取热电联产机组的热能输出
        total_heat_chp = solution[self.dim_P + self.dim_PH : self.dim_P + 2*self.dim_PH]

        # 获取热电联产机组的功率输出
        power_chp = total_power[self.dim_P:self.dim_P + self.dim_PH]
        
        # 使用vmap批量更新每个热电联产机组的热能输出
        total_heat_chp = jax.vmap(lambda i: self.update_chp_unit_heat(power_chp, total_heat_chp, i))(
            jnp.arange(self.dim_PH)
        )


In [ ]:
# 热能平衡约束循环
        def heat_condition(state):
            delta_heat, heat_chp, heat_thermal, key = state
            return jnp.abs(delta_heat) > 1e-2

        # 定义循环体函数
        def heat_body(state):
                delta_heat, heat_chp, heat_thermal, key = state
                # 随机选择一个索引
                key, subkey = jax.random.split(key)
                r = jax.random.randint(subkey, shape=(), minval=0, maxval=self.dim_PH + self.dim_H)
                # 计算一个调整量而不是每次都选择使用这个差值进行计算
                adjustment_heat = delta_heat / (self.dim_PH + self.dim_H)
        
               # 根据索引选择热能机组类型并调整
                def adjust_chp_heat():
                    new_heat_chp = heat_chp.at[r].set(
                        jnp.clip(
                            heat_chp[r] - adjustment_heat, 
                            0.0,  # 热电联产机组热能输出下限
                            self.update_chp_unit_heat(
                                total_power[self.dim_P + r:self.dim_P + r + 1], 
                                heat_chp[r:r+1]
                            )[0]  # 热电联产机组热能输出上限
                        )
                    )
                    return new_heat_chp, heat_thermal
                
                def adjust_thermal_heat():
                    new_heat_thermal = heat_thermal.at[r - self.dim_PH].set(
                        jnp.clip(
                            heat_thermal[r - self.dim_PH] - adjustment_heat, 
                            self.Hmin[r - self.dim_PH], 
                            self.Hmax[r - self.dim_PH]
                        )
                    )
                    return heat_chp, new_heat_thermal
                
                # 根据索引选择热能机组类型
                heat_chp, heat_thermal = jax.lax.cond(
                    r < self.dim_PH, 
                    adjust_chp_heat, 
                    adjust_thermal_heat
                )
                
                # 更新热能差值
                delta_heat = jnp.sum(heat_chp) + jnp.sum(heat_thermal) - self.Hd
                
                return delta_heat, heat_chp, heat_thermal, key
            
        # ## 执行循环确保功率平衡
        delta_heat, total_heat_chp, total_heat_thermal, key = jax.lax.while_loop(
            heat_condition,
            heat_body,
            (delta_heat, total_heat_chp, total_heat_thermal, key)
        )

        # 将更新后的热能输出写回解中# 更新解
        solution = solution.at[heat_chp_indices].set(total_heat_chp)
        solution = solution.at[heat_thermal_indices].set(total_heat_thermal)

In [ ]:
# new_heat_r = jax.lax.cond(
                #     r < self.dim_PH,
                #     lambda h: jnp.clip(h, 0.0, self.update_chp_unit_heat(
                #         total_power[self.dim_P + r:self.dim_P + r + 1], 
                #         heat_chp[r:r+1]
                #     )[0]),
                #     lambda h: jnp.clip(h, self.Hmin[r - self.dim_PH], self.Hmax[r - self.dim_PH]),
                #     new_heat_r
                # )
                ## 上一个版本的写法
                # new_heat_r = jax.lax.cond(

                #     r < self.dim_PH,
                #     lambda h , _ : jax.lax.cond(
                #         (r == 0) | (r == 2),
                #         # CHP 14/16 的热能约束
                #         lambda h,_: jnp.clip(
                #             h,
                #             jnp.where(
                #                 (total_power[self.dim_P + r] >= 81) & (total_power[self.dim_P + r] <= 98.8),
                #                 104.8 - (524 / 89) * (total_power[self.dim_P + r] - 81),
                #                 0.0
                #             ),
                #             jnp.where(
                #                 (total_power[self.dim_P + r] >= 81) & (total_power[self.dim_P + r] <= 215),
                #                 104.8 + (188 / 335) * (total_power[self.dim_P + r] - 81),
                #                 jnp.where(
                #                     (total_power[self.dim_P + r] > 215) & (total_power[self.dim_P + r] <= 247),
                #                     - (45 / 8) * (total_power[self.dim_P + r] - 247),
                #                     0.0
                #                 )
                #             )
                #         ),
                #         lambda h, _: jax.lax.cond(
                #             (r == 1) | (r == 3),
                #             # CHP 15/17 的热能约束
                #             lambda h, _: jnp.clip(
                #                 h,
                #                 jnp.where(
                #                     (total_power[self.dim_P + r] >= 40) & (total_power[self.dim_P + r] <= 44),
                #                     75 - (591 / 40) * (total_power[self.dim_P + r] - 40),
                #                     0.0
                #                 ),
                #                 jnp.where(
                #                     (total_power[self.dim_P + r] >= 40) & (total_power[self.dim_P + r] <= 110.2),
                #                     75 + (101 / 117) * (total_power[self.dim_P + r] - 40),
                #                     jnp.where(
                #                         (total_power[self.dim_P + r] > 110.2) & (total_power[self.dim_P + r] <= 125.8),
                #                         32.4 - (86 / 13) * (total_power[self.dim_P + r] - 125.8),
                #                         0.0
                #                     )
                #                 )
                #             ),
                #             lambda h, _: jax.lax.cond(
                #                 r == 4,
                #                 # CHP 18 的热能约束
                #                 lambda h, _: jnp.clip(
                #                     h,
                #                     jnp.where(
                #                         (total_power[self.dim_P + r] >= 10) & (total_power[self.dim_P + r] <= 20),
                #                         40 - 4 * (total_power[self.dim_P + r] - 10),
                #                         0.0
                #                     ),
                #                     jnp.where(
                #                         (total_power[self.dim_P + r] >= 10) & (total_power[self.dim_P + r] <= 45),
                #                         40 + (3 / 7) * (total_power[self.dim_P + r] - 10),
                #                         jnp.where(
                #                             (total_power[self.dim_P + r] > 45) & (total_power[self.dim_P + r] <= 60),
                #                             - (11 / 3) * (total_power[self.dim_P + r] - 60),
                #                             0.0
                #                         )
                #                     )
                #                 ),
                #                 # CHP 19 的热能约束
                #                 lambda h, _: jnp.clip(
                #                     h,
                #                     0.0,
                #                     jnp.where(
                #                         (total_power[self.dim_P + r] >= 35) & (total_power[self.dim_P + r] <= 90),
                #                         20 + (5 / 11) * (total_power[self.dim_P + r] - 35),
                #                         jnp.where(
                #                             (total_power[self.dim_P + r] > 90) & (total_power[self.dim_P + r] <= 105),
                #                             - (5 / 3) * (total_power[self.dim_P + r] - 105),
                #                             0.0
                #                         )
                #                     )
                #                 )
                #             )
                #         )
                #     ),
                #     # 非热电联产机组的热能约束
                #     lambda h, _: jnp.clip(h, self.Hmin[r - self.dim_PH], self.Hmax[r - self.dim_PH]),
                #     new_heat_r
                # )
                ## 使用之前定义的热能调整函数

In [ ]:
def update_chp_unit_heat_all(self, power_chp, heat):
        """
        更新热电联产机组的热能输出

        参数:
            power_chp: 热电联产机组的功率输出数组 (dim_PH,)
            heat: 热能输出数组 (dim_PH,)

        返回:
            更新后的热能输出数组
        """
        # 将功率和热能输出转换为 float32 类型
        p = power_chp.astype(jnp.float32)
        h = heat.astype(jnp.float32)

        # 定义计算热能输出的辅助函数
        def compute_heat(hi, lower_bound, upper_bound):
            return jnp.clip(hi, lower_bound, upper_bound)

        # 定义每个索引对应的处理逻辑
        def compute_adjusted_heat(index):
            pi = p[index]
            hi = h[index]

            def chp_14_16():
                cond1 = (pi >= 81) & (pi <= 98.8)
                cond2 = (pi > 98.8) & (pi <= 215)
                cond3 = (pi > 215) & (pi <= 247)

                lower_bound = jnp.where(cond1, 104.8 - (524 / 89) * (pi - 81), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    104.8 + (188 / 335) * (pi - 81),
                    jnp.where(cond3, - (45 / 8) * (pi - 247), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_15_17():
                cond1 = (pi >= 40) & (pi <= 44)
                cond2 = (pi > 44) & (pi <= 110.2)
                cond3 = (pi > 110.2) & (pi <= 125.8)

                lower_bound = jnp.where(cond1, 75 - (591 / 40) * (pi - 40), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    75 + (101 / 117) * (pi - 40),
                    jnp.where(cond3, 32.4 - (86 / 13) * (pi - 125.8), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_18():
                cond1 = (pi >= 10) & (pi <= 20)
                cond2 = (pi > 20) & (pi <= 45)
                cond3 = (pi > 45) & (pi <= 60)

                lower_bound = jnp.where(cond1, 40 - 4 * (pi - 10), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    40 + (3 / 7) * (pi - 10),
                    jnp.where(cond3, - (11 / 3) * (pi - 60), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_19():
                cond1 = (pi >= 35) & (pi <= 90)
                cond2 = (pi > 90) & (pi <= 105)

                lower_bound = 0.0
                upper_bound = jnp.where(
                    cond1,
                    20 + (5 / 11) * (pi - 35),
                    jnp.where(cond2, - (5 / 3) * (pi - 105), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def default_case():
                return hi

            # 使用 lax.cond 选择对应的处理逻辑
            adjusted_hi = lax.cond(
                (index == 0) | (index == 2),
                chp_14_16,
                lambda: lax.cond(
                    (index == 1) | (index == 3),
                    chp_15_17,
                    lambda: lax.cond(
                        index == 4,
                        chp_18,
                        lambda: lax.cond(
                            index == 5,
                            chp_19,
                            default_case
                        )
                    )
                )
            )
            #return adjusted_hi

        # 使用 jax.vmap 对所有索引进行处理
        adjusted_heat = jnp.array(jax.vmap(compute_adjusted_heat)(jnp.arange(self.dim_PH)))

        return adjusted_heat

In [ ]:
def adjust_heat_balance(self, solution):
    # 定义热能输出部分的索引
    heat_indices = slice(self.dim_P + self.dim_PH, None)
    # 定义热电联产机组热功率部分的索引
    heat_chp_indices = slice(self.dim_P + self.dim_PH, self.dim_P + 2 * self.dim_PH)
    # 定义纯热机组热功率部分的索引
    heat_thermal_indices = slice(self.dim_P + 2 * self.dim_PH, None)

    # 获取所有的热能输出
    total_heat = solution[heat_indices]
    # 获取热电联产机组的热能输出
    heat_chp = solution[heat_chp_indices]
    # 获取纯热机组的热能输出
    heat_thermal = solution[heat_thermal_indices]

    # 获取热电联产机组的功率输出
    power_chp_indices = slice(self.dim_P, self.dim_P + self.dim_PH)
    power_chp = solution[power_chp_indices]

    # 更新热电联产机组的热能输出
    update_heat_chp = self.update_chp_unit_heat_all(power_chp, heat_chp)
    # 更新 solution 数组中的热电联产机组热能输出
    solution = solution.at[heat_chp_indices].set(update_heat_chp)

    # 计算热能差值
    total_heat = jnp.concatenate([update_heat_chp, heat_thermal])
    delta_heat = jnp.sum(total_heat) - self.Hd

    # 定义循环条件
    def heat_condition(state):
        delta_heat, heat, key = state
        return jnp.abs(delta_heat) > 1e-2

    # 定义循环体函数
    def heat_body(state):
        delta_heat, heat, key = state
        # 随机选择一个索引
        key, subkey = jax.random.split(key)
        r = jax.random.randint(subkey, shape=(), minval=0, maxval=self.dim_PH + self.dim_H)
        # 计算一个调整量
        adjustment_heat = delta_heat / (self.dim_PH + self.dim_H)

        # 根据 r 的值选择更新函数
        def update_unit(heat, r, adjustment_heat):
            def update_chp():
                new_heat_r = heat[r] - adjustment_heat
                new_heat_r = self.update_chp_unit_heat_once(power_chp[r], new_heat_r, r)
                return new_heat_r

            def update_thermal():
                index = r - self.dim_PH
                new_heat_r = heat[r] - adjustment_heat
                new_heat_r = jnp.clip(new_heat_r, self.Hmin[index], self.Hmax[index])
                return new_heat_r

            new_heat_r = jax.lax.cond(
                r < self.dim_PH,
                update_chp,
                update_thermal
            )
            return heat.at[r].set(new_heat_r)

        # 更新 heat 数组
        heat = update_unit(heat, r, adjustment_heat)

        # 重新计算 delta_heat
        delta_heat = jnp.sum(heat) - self.Hd

        return delta_heat, heat, key

    # 初始化随机数种子
    key = jax.random.PRNGKey(0)

    # 初始化 total_heat，将热电联产和纯热机组的热能输出合并
    total_heat = jnp.concatenate([update_heat_chp, heat_thermal])

    # 执行循环，调整热能输出以满足热能平衡约束
    delta_heat, total_heat, key = jax.lax.while_loop(
        heat_condition,
        heat_body,
        (delta_heat, total_heat, key)
    )

    # 更新 solution 中的热能输出部分
    solution = solution.at[heat_indices].set(total_heat)

    return solution


In [ ]:
import jax
import jax.numpy as jnp

def adjust_power_balance(self, solution):
    """调整功率输出以满足功率等式约束（确定性方法）"""
    # 定义解数组中功率部分的索引
    power_indices = slice(0, self.dim_P + self.dim_PH)
    # 提取相关的功率部分
    total_power = solution[power_indices]
    # 计算当前总功率与目标功率的差值
    delta_power = jnp.sum(total_power) - self.Pd

    # 定义条件函数，检查 delta_power 是否大于阈值
    def condition(state):
        delta_power, power = state
        return jnp.abs(delta_power) > 1e-2

    # 定义循环体函数
    def body(state):
        delta_power, power = state
        # 计算每个元素的调整量
        adjustment = delta_power / (self.dim_P + self.dim_PH)
        # 对所有功率元素进行调整
        power = power - adjustment
        # 分别处理不同部分的上下限约束
        # 第一部分：dim_P 元素
        power_P = power[:self.dim_P]
        power_P = jnp.clip(power_P, self.Pmin, self.Pmax)
        # 第二部分：dim_PH 元素
        power_PH = power[self.dim_P:]
        power_PH = jnp.clip(power_PH, self.PRmin, self.PRmax)
        # 合并调整后的功率部分
        power = jnp.concatenate([power_P, power_PH])
        # 重新计算 delta_power
        new_delta_power = jnp.sum(power) - self.Pd
        return new_delta_power, power

    # 执行循环确保功率平衡
    delta_power, total_power = jax.lax.while_loop(
        condition,
        body,
        (delta_power, total_power)
    )
    # 将更新后的功率写回解中
    total_power = solution.at[power_indices].set(total_power)
    return total_power


In [ ]:
import jax
import jax.numpy as jnp

def adjust_heat_balance(self, solution):
    # 定义热能输出部分的索引
    heat_indices = slice(self.dim_P + self.dim_PH, None)
    # 定义热电联产机组热功率部分的索引
    heat_chp_indices = slice(self.dim_P + self.dim_PH, self.dim_P + 2 * self.dim_PH)
    # 定义纯热电机组热功率部分的索引
    heat_thermal_indices = slice(self.dim_P + 2 * self.dim_PH, None)

    # 获取所有的热能输出
    heat_chp = solution[heat_chp_indices]
    heat_thermal = solution[heat_thermal_indices]

    # 获取热电联产机组的功率输出
    power_chp_indices = slice(self.dim_P, self.dim_P + self.dim_PH)
    power_chp = solution[power_chp_indices]

    # 更新热电联产机组的热能输出
    update_heat_chp = self.update_chp_unit_heat_all(power_chp, heat_chp)
    # 更新 solution 中的热电联产机组热能输出部分
    solution = solution.at[heat_chp_indices].set(update_heat_chp)

    # 合并热能输出
    total_heat = jnp.concatenate([update_heat_chp, heat_thermal])
    # 计算热能差值
    delta_heat = jnp.sum(total_heat) - self.Hd

    # 定义条件函数，检查 delta_heat 是否大于阈值
    def heat_condition(state):
        delta_heat, heat = state
        return jnp.abs(delta_heat) > 1e-2

    # 定义循环体函数
    def heat_body(state):
        delta_heat, heat = state
        total_units = self.dim_PH + self.dim_H  # 总的热能输出单元数量
        # 计算每个单元的调整量
        adjustment_heat = delta_heat / total_units

        # 分别调整热电联产机组和纯热机组
        # 调整热电联产机组
        heat_chp = heat[:self.dim_PH]
        power_chp = solution[power_chp_indices]
        indices_chp = jnp.arange(self.dim_PH)

        def adjust_chp_unit(p, h, idx):
            new_heat = h - adjustment_heat
            new_heat = self.update_chp_unit_heat_once(p, new_heat, idx)
            return new_heat

        # 向量化调整热电联产机组的热能输出
        heat_chp_updated = jax.vmap(adjust_chp_unit, in_axes=(0, 0, 0))(power_chp, heat_chp, indices_chp)

        # 调整纯热机组
        heat_thermal = heat[self.dim_PH:]
        heat_thermal_adjusted = heat_thermal - adjustment_heat
        heat_thermal_updated = jnp.clip(heat_thermal_adjusted, self.Hmin, self.Hmax)

        # 合并更新后的热能输出
        heat_updated = jnp.concatenate([heat_chp_updated, heat_thermal_updated])

        # 重新计算 delta_heat
        delta_heat = jnp.sum(heat_updated) - self.Hd

        return delta_heat, heat_updated

    # 初始化循环状态
    state = (delta_heat, total_heat)

    # 执行循环，调整热能输出以满足热能平衡约束
    delta_heat, total_heat = jax.lax.while_loop(
        heat_condition,
        heat_body,
        state
    )

    # 更新 solution 中的热能输出部分
    solution = solution.at[heat_indices].set(total_heat)

    return solution
